## Using raw assistant APIs


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
chatServiceName = "chatService"
supporterName = "Supporter"
opponentName = "Opponent"
supporterInstructions = """You are the supporter agent supporting the user proposal. You should only articulate reasons supporting the user proposal. Do not provide any reasons for the proposition being false or not acceptable.
    You should also argue against any arguments raised by the opponent agent and listed in the history.
    Be brief and concise in presenting your arguments.
    If you cannot find a good argument respond with 'I give up.'"""
opponentInstructions = """You are the opponent agent whose job is to argue against the User proposal and against the supporter agents arguments. Do not provide any reasons for the proposition being true or acceptable.
    Be brief and concise in presenting your arguments.
    If you cannot find a good argument respond with 'I give up.'"""

In [8]:
import os
from openai import AzureOpenAI

client = AzureOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],    
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview",
)

# Create assistants
supporter = client.beta.assistants.create(
    name=supporterName,
    instructions=supporterInstructions,
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),  
)
opponent = client.beta.assistants.create(
    name=opponentName,
    instructions=opponentInstructions,
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),  
)

In [9]:
thread = client.beta.threads.create()
user_input = input("User:> ")
print(f"## Proposal\n\n{user_input}\n\n")
turns = 0
if user_input:
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content= f"Proposal: {user_input}"
    )
    is_complete: bool = False
    agents = [ supporter, opponent ]
    agentNo = 0
    while not is_complete:
        agent = agents[agentNo]
        # Add a new instruction to the existing one
        oppose = f".In your response focus on rebutting arguments of the {agents[(agentNo + 1) % 2].name}" if turns > 0 else None
        run = client.beta.threads.runs.create_and_poll(
            thread_id=thread.id,
            assistant_id=agent.id,
            additional_instructions=f"Prefix your response with '{agent.name}:'. {oppose}",
        )
        if run.status == "completed":
            messages = client.beta.threads.messages.list(thread_id=thread.id)
            #print(messages.to_json(indent=2))
            response = messages.data[0].content[0].text.value
            print(f"## {agents[agentNo].name}\n\n{response}\n\n")
        if response == "I give up.":
            is_complete = True
            break
        turns += 1
        if turns == 6: # Should be even to allow both agents to respond
            print("## Timeout!!!")
            is_complete = True
            break
        agentNo = (agentNo + 1) % 2


## Proposal

End justifies the means


## Supporter

Supporter: The phrase "the end justifies the means" suggests that actions taken to achieve a desired outcome can be rationalized if the outcome itself is considered valuable or necessary. This concept has been discussed and debated throughout history, particularly in the realms of politics, ethics, and philosophy. Here are some points supporting this proposition:

1. **Pragmatism**: In many scenarios, achieving a greater good or a critical goal may require taking actions that are controversial or deemed immoral. For example, in times of war, leaders might implement strategies that sacrifice individual rights for the sake of national security.

2. **Moral Flexibility**: Life often presents complex situations where strict adherence to ethical codes may hinder progress or improvement. Recognizing that moral absolutes may need to be adjusted can lead to innovative solutions to problems.

3. **Historical Precedent**: Throughout history, m